In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import operator
from infomap import infomap
import community
from pymongo import MongoClient
%matplotlib inline

Reading the graphml file

In [2]:
filename = "/Users/arefindk/Desktop/github_merged_pull/trimmed_merged_pull_users_strong_int.graphml"
G = nx.read_graphml(filename)

In [3]:
def findCommunities(G,directed = False):
    """
    Partition network with the Infomap algorithm.
    Annotates nodes with 'community' id and return number of communities found.
    """
    if directed:
        conf = infomap.init("--directed");
    else:
        conf = infomap.init("--undirected");
    # Input data
    network = infomap.Network(conf);
    # Output data
    tree = infomap.HierarchicalNetwork(conf)

    print("Building network...")
    for e in G.edges_iter():
        network.addLink(int(e[0]), int(e[1]))

    network.finalizeAndCheckNetwork(True, nx.number_of_nodes(G));

    # Cluster network
    infomap.run(network, tree);

    print("Found %d top modules with codelength: %f" % (tree.numTopModules(), tree.codelength()))

    communities = {}
    clusterIndexLevel = 1 # 1, 2, ... or -1 for top, second, ... or lowest cluster level
    for node in tree.leafIter(clusterIndexLevel):
        communities[str(node.originalLeafIndex)] = node.clusterIndex()

    nx.set_node_attributes(G, 'infomap_community', communities)
    return tree.numTopModules()

In [4]:
numCommunities = findCommunities(G,directed=True)

Building network...
Found 93 top modules with codelength: 9.507665


In [5]:
ug = G.to_undirected()

In [6]:
partition = community.best_partition(ug)

In [7]:
print len(set(partition.values()))

74


In [8]:
nx.set_node_attributes(G,'louvain_community',partition)

In [9]:
for x in xrange(10):
    x = str(x)
    print G.node[x]['lang'], G.node[x]['infomap_community'],G.node[x]['louvain_community']

Shell 10 1
JavaScript 1 21
Java 5 1
C# 1 8
Perl 0 18
Ruby 0 2
Ruby 0 2
PHP 3 3
Java 37 7
JavaScript 1 8


In [10]:
nx.write_graphml(G,"trimmed_merged_pull_users_strong_int_with_communities.graphml")